In [ ]:
from sys import path as wd
from os import path, getcwd
from tqdm import tqdm
from time import sleep

workdir = path.dirname(getcwd())
wd.append(workdir)

from dags.operadores import (GoogleSheetManager, GoogleDriveManager)
from dags.estruturas import (InformationManager)
from dags.sidrapi import (SidraManager)

import json
import pandas as pd

In [ ]:
data_json = path.join(workdir, "data", "__data__info.json")
local_dir = path.join(workdir, "data", "processed")

with open(data_json, 'r') as file:
    info_data = json.load(file)

In [ ]:
# Nomeclaturas 
COLUMN_NAME = "tabela" # coluna com o nome da tabela
TABLE_INDEX = 0

In [ ]:
credentials_dir = path.join(workdir, "data", "credentials.json")
folder_name = info_data["MainFolder"]["Name"]
knowledge_sheet_url = r'https://docs.google.com/spreadsheets/d/11-CI4nLXTte0OcDVMl-My4PtOdif7ziWB0GKg6hkJMw/edit?usp=sharing'#info_data["Knowledge base"]

# Classes
sheet_manager = GoogleSheetManager(credentials_dir)
sidra_manager = SidraManager()

In [ ]:
df_information = sheet_manager.get_data_info_from_url(knowledge_sheet_url)

In [ ]:
sidra_manager.sidra_get_metadata(5436)

In [ ]:
# nova versão

# Lista para armazenar os DataFrames
lista_df_variaveis = pd.DataFrame()
lista_df_categorias = pd.DataFrame()


for idx, row in tqdm(df_information.iterrows(), total=df_information.shape[0], unit="Tabelas"):
    table_number = row[COLUMN_NAME]
    dados = sidra_manager.sidra_get_metadata(table_number)
    sleep(1)
    if dados:
        request_id = row[COLUMN_NAME]
        df_variaveis = sidra_manager.sidra_process_variables(dados, request_id)
        df_categorias = sidra_manager.sidra_process_categories(dados, request_id)

        # Adicionando os DataFrames às respectivas listas
        lista_df_variaveis = pd.concat([lista_df_variaveis, df_variaveis], ignore_index=True)    # lista_df_variaveis.append(df_variaveis)
        lista_df_categorias = pd.concat([lista_df_categorias, df_categorias], ignore_index=True)    # lista_df_categorias.append(df_categorias)

    else:
        print("Falha ao obter dados.")

sidra_manager.retry_failed_requests() 

In [ ]:
lista_df_categorias

In [ ]:
lista_df_variaveis

In [ ]:
lista_df_variaveis.to_excel(f"{local_dir}/variaveis_ajustadas.xlsx", index=False)
lista_df_categorias.to_excel(f"{local_dir}/categorias_ajustadas.xlsx", index=False)

In [ ]:
if False: 
    # Initialize empty DataFrames
    df_tabelas_concatenados = pd.DataFrame()
    df_variaveis_concatenados = pd.DataFrame()
    df_grupos_concatenados = pd.DataFrame()

    # Iterate through rows in df_information
    for idx, row in tqdm(df_information.iterrows(), total=df_information.shape[0]):
        table_number = row[COLUMN_NAME]

        try:
            # Get data using sidra_manager.sidra_info
            (api_content, df_table) = sidra_manager.sidra_info(table_number)
            (df_variables, df_groups) = sidra_manager.sidra_get_vars(api_content)

            # Concatenate tables
            df_tables_expanded = pd.json_normalize(df_table.to_dict('records'))
            df_tables_expanded[COLUMN_NAME] = table_number
            df_tabelas_concatenados = pd.concat([df_tabelas_concatenados, df_tables_expanded], ignore_index=True)

            # Treat and concatenate variables
            df_variables_expanded = pd.json_normalize(df_variables.to_dict('records')) if df_variables is not None else pd.DataFrame()
            df_variables_expanded[COLUMN_NAME] = table_number
            df_variaveis_concatenados = pd.concat([df_variaveis_concatenados, df_variables_expanded], ignore_index=True)

            # Treat and concatenate groups
            df_groups_expanded = pd.json_normalize(df_groups.to_dict('records')) if df_variables is not None else pd.DataFrame()
            df_groups_expanded[COLUMN_NAME] = table_number
            df_grupos_concatenados = pd.concat([df_grupos_concatenados, df_groups_expanded], ignore_index=True)

            sleep(1)  # delay

        except Exception as e:
            print(f"An error occurred for table number {table_number}: {e}")
            sleep(10)
            continue  

    # Saving to Excel
    try:
        with pd.ExcelWriter(f'{local_dir}\dados_sidra.xlsx', engine='openpyxl') as writer:
            df_tabelas_concatenados.to_excel(writer, sheet_name='Tabelas', index=False)
            df_variaveis_concatenados.to_excel(writer, sheet_name='Variáveis', index=False)
            df_grupos_concatenados.to_excel(writer, sheet_name='Grupos', index=False)
    except Exception as e:
        print(f"An error occurred while saving to Excel: {e}")

In [ ]:
if False:
    import re
    # test = [item.split("(")[0] for item in df_tabelas_concatenados['Tipo do Período'].tolist()]
    # test2 = [re.findall(r'\((.*?)\)', item)[0] for item in df_tabelas_concatenados['Tipo do Período'].tolist()]

    test3 = df_tabelas_concatenados.copy()
    test3['Número de Períodos'] = test3['Tipo do Período'].apply(lambda x: re.findall(r'\((.*?)\)', x)[0] if re.findall(r'\((.*?)\)', x) else None)
    test3['Tipo do Período'] = test3['Tipo do Período'].apply(lambda x: x.split("(")[0] if "(" in x else "")